# Object Pools

Let's start with a simple sketch, without worrying about type safety. All we really need, here, is a queue of objects. But when we pull objects from this pool, they need to come equipped with some means of rejoining the pool.

In [1]:
using DataStructures

In [2]:
mutable struct UnsafePool{T}
    q::Queue{T}
    UnsafePool{T}() where {T} = new(Queue{T}())
end

To do this, we'll wrap them in a struct that contains both the object and a reference back to the pool from whence it came.

In [3]:
mutable struct Recyclable{T}
    inner::Union{Nothing, T}
    home::Union{Nothing,Ref{UnsafePool{T}}}
end

In [4]:
function pull!(pool::UnsafePool{T})::Recyclable{T} where {T}
    item = dequeue!(pool.q)
    recyc = Recyclable{T}(item, Ref{UnsafePool{T}}(pool))
    return recyc
end

pull! (generic function with 1 method)

In [5]:
# for example
u = UnsafePool{Int}()
enqueue!(u.q, 1)
enqueue!(u.q, 2)
enqueue!(u.q, 3)
enqueue!(u.q, 4)
u

UnsafePool{Int64}(Queue{Int64}(Deque [[1, 2, 3, 4]]))

In [6]:
recyc = pull!(u)

Recyclable{Int64}(1, Base.RefValue{UnsafePool{Int64}}(UnsafePool{Int64}(Queue{Int64}(Deque [[2, 3, 4]]))))

In [7]:
function recycle!(r::Recyclable{T}) where {T}
    if r.home === nothing
        error("Already recycled $r")
    end
    enqueue!(r.home[].q, r.inner)
    r.home = nothing
    r.inner = nothing
    return
end

recycle! (generic function with 1 method)

In [8]:
# let's look at recyc, the wrapped object we pulled from the pool
println(recyc)

Recyclable{Int64}(1, Base.RefValue{UnsafePool{Int64}}(UnsafePool{Int64}(Queue{Int64}(Deque [[2, 3, 4]]))))


In [9]:
# and here's the pool it came from
println(u)

UnsafePool{Int64}(Queue{Int64}(Deque [[2, 3, 4]]))


In [10]:
# maybe we do some work on recyc
recyc.inner += 1
println(recyc)

Recyclable{Int64}(2, Base.RefValue{UnsafePool{Int64}}(UnsafePool{Int64}(Queue{Int64}(Deque [[2, 3, 4]]))))


In [11]:
# now we're done with it, so we recycle it
println("recyc = $recyc")
println("pool before recycling: $u")
recycle!(recyc)
println("pool after recycling: $u")



recyc = Recyclable{Int64}(2, Base.RefValue{UnsafePool{Int64}}(UnsafePool{Int64}(Queue{Int64}(Deque [[2, 3, 4]]))))
pool before recycling: UnsafePool{Int64}(Queue{Int64}(Deque [[2, 3, 4]]))
pool after recycling: UnsafePool{Int64}(Queue{Int64}(Deque [[2, 3, 4, 2]]))


In [12]:
println("recyc after recycling (throw this away): $recyc")

recyc after recycling (throw this away): Recyclable{Int64}(nothing, nothing)


## Let's try this with non-primitive objects now

In [13]:
array_pool = UnsafePool{Array{Float64,2}}()

UnsafePool{Array{Float64,2}}(Queue{Array{Float64,2}}(Deque [Array{Float64,2}[]]))

In [14]:
# now let's fill it up with a few expensively produced objects
for i in 1:4
    enqueue!(array_pool.q, rand(10, 10))
end
println(array_pool)

UnsafePool{Array{Float64,2}}(Queue{Array{Float64,2}}(Deque [[[]]]))


In [15]:
# take a recyclable container from the pool\
ar = pull!(array_pool)
println("remaining in pool: $(length(array_pool.q))")
# do some work on it
println("max val in ar is $(maximum(ar.inner))")
# now return it to the pool
recycle!(ar)
println("remaining in pool: $(length(array_pool.q))")
println("ar = $ar")

remaining in pool: 3
max val in ar is 0.9859517269930667
remaining in pool: 4
ar = Recyclable{Array{Float64,2}}(nothing, nothing)


## Something approximating Python's `with` syntax

It would be nice if we didn't have to worry about recycling our pool items directly, and could just let the Object Pool abstraction layer take care of that kind of thing. Here, we'll try to implement something similar to Python's `with` syntax...

In [19]:
function from_pool(f::Function, pool::Ref{UnsafePool{T}}) where {T}
    r = pull!(pool[])
    try
        result = f(r.inner)
        return result
    finally
        println("recycling...")
        recycle!(r)
    end
end


from_pool (generic function with 1 method)

In [26]:
from_pool(Ref(array_pool)) do array
    println("The maximum value in this array is $(maximum(array))")
end
    

The maximum value in this array is 0.9962407190513243
recycling...


## Thread Safety

So far, we've been playing around with an unsafe model of our Object Pool, but this sort of data structure isn't much use to us if we can't access it safely across threads.